# Лабораторная работа 1. Подсчет ошибки распознавания

WER (Word Error Rate) является метрикой для оценки качества систем распознавания речи. Она показывает процент ошибочных слов в гипотезе распознавания по сравнению с эталонным текстом. WER учитывает три типа ошибок: вставки, удаления и замены. 
$$ WER = {I + D + S \over D + S + C} $$
где I, D, S - количество втавок, удалений и замен, соответственно. C - количество правильно распознанных слов

Лабораторная работа состоит из трех частей. Первая часть (функция подсчета WER) обязательная, остальные дополнительные. Всего за работу можно получить максимум 20 баллов. 4 за сдачу в срок и 16 за задания: 
* функция подсчета WER (тест 1.a, 1.b) - 8 баллов
* функция подсчета WER и ошибки пунктуации (тест 2.a) - 4 балла
* функция подсчета SA-WER (тест 3.а) - 4 балла

# 1. Word Error Rate (8 баллов)

## 1.a. подсчет WER 


Функция должна принимать две строки в качестве входных данных: эталонный текст и распознанный текст. Эталонный текст - это то, что произносится в аудиозаписи, а гипотеза распознавания - это текст, полученный от системы распознавания речи. Для корректного вычисления ошибки распознавания необходимо удалить все символы пунктуации и привести все слова к нижнему регистру.



In [1]:
import re

In [2]:
def calculate_wer(reference_text: str, recognized_text: str) -> float:
    # Приведение текста к нижнему регистру, удаление символов пунктуации и разбивка на слова
    # TODO

    reference_words = re.findall('[0-9А-ЯA-Zа-яa-z]+', reference_text)
    reference_words = [word.lower() for word in reference_words]
    #print(reference_words)

    recognized_words = re.findall('[0-9А-ЯA-Zа-яa-z]+', recognized_text)
    recognized_words = [word.lower() for word in recognized_words]
    #print(recognized_words)
    
    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i-1] == recognized_words[j-1]:
                cost = 0
            else:
                cost = 1
            
            distance_matrix[i][j] = min(distance_matrix[i - 1][j] + 1, 
                                        distance_matrix[i][j - 1] + 1, 
                                        distance_matrix[i - 1][j - 1] + cost) 
                

    # Расчет WER  (в процентах)
    #print(distance_matrix)
    wer = distance_matrix[-1][-1]*100/len(reference_words)
    return wer

wer = calculate_wer('Я ел солонину', 'Я ел слона')
print(f"Word Error Rate: {wer:.2f}%")

Word Error Rate: 33.33%


In [3]:
def assert_wer(ref, hyp, ideal_wer):
    wer = calculate_wer(ref, hyp)
    assert round(wer, 2) == round(ideal_wer, 2), f"for '{hyp=}' and '{ref=}' {ideal_wer=}, calculate_wer {wer=}"
    
def test_wer():
    assert_wer('привет студент', 'привет студент', 0)
    assert_wer('привет! Студент.', 'Привет, студент?', 0)
    assert_wer('привет студент', 'студент', 50)
    assert_wer('привет студент', '', 100)
    assert_wer('привет студент', 'студент привет', 100)
    assert_wer('привет', 'привет студент', 100)
    assert_wer('привет студент привет как дела', 'студент привет', 60)
    assert_wer('привет студент привет как дела', 'привет как дела', 40)
    assert_wer('привет студент привет как дела ', 'привет студент дела ', 40)
    assert_wer('привет студент привет как дела '*100, 'привет студент дела '*100, 40)

    print(f"Test 1.a passed")
    
test_wer() 
    

Test 1.a passed


In [4]:
#источники
#https://skine.ru/articles/411736/
#https://habr.com/ru/articles/114997/

## 1.b. Построение выравнивания
Реализованная в части 1.a. функция выдает только суммарное значение ошибки распознавания, не давая понимания, в чем состоят основные проблемы распознавания. 

Значение WER получается из трех видов ошибок: 
* вставка (insertion)
* удаление (deletion)
* замена (substitution)

Каждый тип ошибок имеет свое значение и указывает на определенные недостатки системы. Например, большое количество вставок означает, что ASR слышит речь там где ее нет. А большое количество удалений показывает, что целевая речь пропускается и не транскрибируется. 

Кроме числовых значений каждой ошибки, для анализа результатов работы системы может пригодиться выравнивание эталонной текстовки и гипотезы распознавания относительно друг друга. 

пример выравнивания: 

```
>>> tabulate(ali)

Я сегодня  ***   учуcь  в  универе
Я    с    завтра учусь *** универе  
C    S      I      C    D    C    
```

Реализуйте функцию, которая кроме числового значения WER возвращает выравнивание, а также значения каждого типа ошибок распознавания (вставки, удаления, замены).

In [5]:
from tabulate import tabulate
# используйте tabulate для отладки

In [93]:
def calculate_wer_with_alignment(reference_text: str, recognized_text: str):
    
    # Перенесите сюда код из задания 1.a.
    reference_words = re.findall('[0-9А-ЯA-Zа-яa-z]+', reference_text)
    reference_words = [word.lower() for word in reference_words]
    #print(reference_words)

    recognized_words = re.findall('[0-9А-ЯA-Zа-яa-z]+', recognized_text)
    recognized_words = [word.lower() for word in recognized_words]
    #print(recognized_words)
    
    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    #print(distance_matrix)
    
    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i-1] == recognized_words[j-1]:
                cost = 0
            else:
                cost = 1
            distance_matrix[i][j] = min(distance_matrix[i - 1][j] + 1, 
                                        distance_matrix[i][j - 1] + 1, 
                                        distance_matrix[i - 1][j - 1] + cost) 
                

    # Расчет WER  (в процентах)
    #print(distance_matrix)
    wer = distance_matrix[-1][-1]*100/len(reference_words)
    #print(wer)
    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 
    #TODO
    correct, deletion, insertion, substitution = 0, 0, 0, 0
    fin_length = max(len(reference_words), len(recognized_words))
    ali = [['***' for i in range(fin_length)],
           ['***' for i in range(fin_length)],
           ['C' for i in range(fin_length)]]
    #print(tabulate(ali))
    k = fin_length-1
    i = len(reference_words)
    j = len(recognized_words)
    
    while i > 0 or j > 0:
        min_dis = distance_matrix[i][j]
        
        if reference_words[i-1] == recognized_words[j-1]:
            correct += 1
            ali[0][k] = reference_words[i-1]
            ali[1][k] = recognized_words[j-1]
            #ali[2][k] = 'C'
        elif (distance_matrix[i - 1][j-1] + 1 == min_dis):
            substitution += 1
            ali[0][k] = reference_words[i-1]
            ali[1][k] = recognized_words[j-1]
            ali[2][k] = 'S'
        elif (distance_matrix[i - 1][j] + 1 == min_dis):
            deletion += 1
            ali[0][k] = reference_words[i - 1]
            #ali[1][k] = recognized_words[k]
            ali[2][k] = 'D'
        else:
            insertion +=1
            #ali[0][k] = reference_words[k]
            ali[1][k] = recognized_words[j-1]
            ali[2][k] = 'I'

        i -= 1
        j -= 1
        k -= 1
        #print(tabulate(ali))
            
    
    # ali[0]=  разбитый по словам референс. Втавки отабражаются в эталонном выравнивании с помощью "***"
    # ali[1] = разбитая по словам гипотеза.
    # ali[2] = аннотация 
    assert len(ali[0]) == len(ali[1]) == len(ali[2]), f"wrong ali {ali}"
    
    return {"wer" : wer,
            "cor": correct, 
            "del": deletion,
            "ins": insertion,
            "sub": substitution,
            "ali": ali}


In [94]:
'''assert_wer_with_alignment('привет студент', 'студент', {
     19             "wer" : 50,
     20             "cor": 1, 
     21             "del": 1,
     22             "ins": 0,
     23             "sub": 0,
     24             "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})'''

calculate_wer_with_alignment('привет студент', 'студент')

{'wer': 50.0,
 'cor': 1,
 'del': 1,
 'ins': 0,
 'sub': 0,
 'ali': [['привет', 'студент'], ['***', 'студент'], ['D', 'C']]}

In [95]:
def assert_wer_with_alignment(ref, hyp, ideal_report):
    report = calculate_wer_with_alignment(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_with_alignment():
    assert_wer_with_alignment('привет студент', 'привет студент',  {
            "wer" : 0,
            "cor": 2, 
            "del": 0,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["привет", "студент"],['C', 'C']]})
    assert_wer_with_alignment('привет студент', 'студент', {
            "wer" : 50,
            "cor": 1, 
            "del": 1,
            "ins": 0,
            "sub": 0,
            "ali": [["привет", "студент"],["***", "студент"],['D', 'C']]})
    assert_wer_with_alignment('привет', 'привет студент', {
            "wer" : 100,
            "cor": 1, 
            "del": 0,
            "ins": 1,
            "sub": 0,
            "ali": [["привет", "***"],["привет", "студент"],['C', 'I']]})
    assert_wer_with_alignment('привет студент', 'пока студент',  {
            "wer" : 50,
            "cor": 1, 
            "del": 0,
            "ins": 0,
            "sub": 1,
            "ali": [["привет", "студент"],["пока", "студент"],['S', 'C']]})

    print(f"Test 1.b passed")
    
test_wer_with_alignment() 

Test 1.b passed


# 2. WER с пунктуацией (4 балла)
Попробуйте модифицировать WER таким образом, чтобы получившаяся метрика учитавала ошибки расстановки знаков препинания. 

Для этого надо ввести ограничение в алгоритм подсчета distance_matrix таким образом, чтобы запретить делать замену знака препинания на слово и наоборот.

Пример выравнивания 
```
Я сегодня  .   ***   ***  А ты  
Я    с    *** завтра  ?   А ты  
C    S    D_p   I    I_p  C  C    
```
Здесь суффикс _p в аннотации к ошибкам означает **ошибки пунктуации**


Задание: 
Напишите функцию, которая кроме стандартного WER считает дополнительно PunctuaionErrorRate (PER) по формуле

$$ PER = {I_p + D_p + S_p \over D_p + S_p + C_p} $$


In [101]:
import nltk
nltk.download('punkt')
from nltk import word_tokenize
from string import punctuation 

punctuations = list(punctuation)

[nltk_data] Downloading package punkt to /Users/sophia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [221]:
def calculate_wer_per(reference_text: str, recognized_text: str):
    # TODO   

    #reference_words = word_tokenize(reference_text) 
    reference_words = nltk.wordpunct_tokenize(reference_text)
    reference_words = [word.lower() for word in reference_words]
    #print(reference_words)

    #recognized_words = word_tokenize(recognized_text)
    recognized_words = nltk.wordpunct_tokenize(recognized_text)
    recognized_words = [word.lower() for word in recognized_words]
    #print(recognized_words)
    
    # расстояние Левенштейна 
    
    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    #print(distance_matrix)
    
    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i-1] == recognized_words[j-1]:
                cost = 0
            else:
                cost = 1
            if (reference_words[i-1] in punctuations and recognized_words[j-1] not in punctuations or 
                recognized_words[j-1] in punctuations and reference_words[i-1] not in punctuations):
                sub_ok = 0
            else:
                sub_ok = 1
            
            distance_matrix[i][j] = min(distance_matrix[i - 1][j] + 1, 
                                        distance_matrix[i][j - 1] + 1, 
                                        distance_matrix[i - 1][j - 1] + cost if sub_ok else 999) 
                

    # Расчет WER  (в процентах)
    #print(distance_matrix)
    #wer = distance_matrix[-1][-1]*100/len(reference_words)
    #print(wer)
    # используя distance_matrix восстановите путь (набор операций), который соответстует найденому WER 

    correct, deletion, insertion, substitution = 0, 0, 0, 0
    correct_p, deletion_p, insertion_p, substitution_p = 0, 0, 0, 0

    '''fin_length = max(len(reference_words), len(recognized_words))
    ali = [['***' for i in range(fin_length)],
           ['***' for i in range(fin_length)],
           ['*' for i in range(fin_length)]]'''

    ali = [[],[],[]]
    #print(tabulate(ali))
    #k = fin_length-1
    i = len(reference_words)
    j = len(recognized_words)
    
    while i > 0 or j > 0:
        min_dis = distance_matrix[i][j]
        
        if reference_words[i-1] == recognized_words[j-1] and (i>0 and j>0):
            if reference_words[i-1] in punctuation:
                correct_p += 1
                ali[2].append('C_p')
            else:
                correct += 1
                ali[2].append('C')
            ali[0].append(reference_words[i-1])
            ali[1].append(recognized_words[j-1])
            i -= 1
            j -= 1
            
        elif (distance_matrix[i - 1][j-1] + 1 == min_dis) and (i>0 and j>0):
            if (reference_words[i-1] in punctuation) or (recognized_words[j-1] in punctuation):
                substitution_p += 1
                ali[2].append('S_p')
            else:
                substitution += 1
                ali[2].append('S')
            ali[0].append(reference_words[i-1])
            ali[1].append(recognized_words[j-1])
            i -= 1
            j -= 1
            
        elif (distance_matrix[i - 1][j] + 1 == min_dis) and i>0:
            if (reference_words[i-1] in punctuation):
                deletion_p += 1
                ali[2].append('D_p')
            else:
                deletion += 1
                ali[2].append('D')
            ali[0].append(reference_words[i - 1])
            ali[1].append('***')
            i -= 1
        
        else:
            if (recognized_words[j-1] in punctuation) and j>0:
                insertion_p +=1
                ali[2].append('I_p')
            else:
                insertion +=1
                ali[2].append('I')
            ali[0].append('***')
            ali[1].append(recognized_words[j-1])
            j -= 1
        #print(i, j)
        #print(ali)

    wer = (substitution+deletion+insertion)*100/(substitution+deletion+correct)
    per = (substitution_p+deletion_p+insertion_p)*100/(substitution_p+deletion_p+correct_p)

    

    return {"wer" : wer,
            "per": per}
            #, 
            #"ali": ali}


In [218]:
calculate_wer_per('привет студент.', '.студент?')

['привет', 'студент', '.']
['.', 'студент', '?']
2 2
[['.'], ['?'], ['S_p']]
1 1
[['.', 'студент'], ['?', 'студент'], ['S_p', 'C']]
0 1
[['.', 'студент', 'привет'], ['?', 'студент', '***'], ['S_p', 'C', 'D']]
0 0
[['.', 'студент', 'привет', '***'], ['?', 'студент', '***', '.'], ['S_p', 'C', 'D', 'I_p']]


{'wer': 50.0,
 'per': 200.0,
 'ali': [['.', 'студент', 'привет', '***'],
  ['?', 'студент', '***', '.'],
  ['S_p', 'C', 'D', 'I_p']]}

In [222]:
def assert_wer_per(ref, hyp, ideal_report):
    report = calculate_wer_per(ref, hyp)
    for k, v in ideal_report.items():
        if isinstance(v, float):
            assert round(v, 2) == round(report[k], 2), f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"
        else:
            assert v == report[k], f"for '{hyp=}' and '{ref=}' {ideal_report=}, calculate_wer {report=}"

    
def test_wer_per():
    assert_wer_per('привет студент.', 'привет студент',  {
            "wer" : 0,
            "per": 100})
    assert_wer_per('привет студент.', 'студент.', {
            "wer" : 50,
            "per": 0,})
    assert_wer_per('привет студент.', 'привет. студент',  {
            "wer" : 0,
            "per": 200})
    assert_wer_per('привет студент.', '.студент?', {
            "wer" : 50,
            "per": 200, })

    print(f"Test 2 passed")
    
test_wer_per() 

Test 2 passed


# 3. Speaker-attributed Word Error Rate (4 балла)

В задаче распознавания диалоговых данных, когда говорят два диктора, важно не только распознать правильно каждое слово, но и отнести его к правильному диктору. Чаще всего такой результат получается с помощью комбинации двух независимых систем: распознавания речи и диаризация. 

При подсчете ошибки распознавания диалоговых систем в формулу WER добавляется еще один тип ошибки - S_I (speaker incorrect).

$$ SA{\text -}WER = \min{I + D + S + S_I \over D + S + C + S_I} $$

Кроме подсчета самой ошибки, sa-wer решает еще одну задачку - поиск маппинга из эталонных названий спикеров (например, имен) в предсказанные (чаще всего Idшники). Это необходимо, тк система диаризации не знает, какие названия у спикеров в эталоне. При подсчете SA-WER проверяются все возможные мапинги спикеров и выбирается тот, который соответствует минимальному значению ошибки. 



In [337]:
def calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers):
    # В отличие от прошлых функций на вход sawer подаются уже разбитые на слова произнесения
    # Кроме списка слов, дополнительно передается список меток спикеров
    assert isinstance(reference_text, list)
    assert isinstance(recognized_text, list)
    assert len(reference_text) == len(reference_speakers)
    assert len(recognized_text) == len(recognized_speakers)

    reference_words = reference_text
    recognized_words = recognized_text
    
    # TODO  посчитайте sawer с учетом мапинга спикеров
    # для этого посчитайте значение ошибки для каждого варианта мапинга меток дикторов 
    # и выберете тот, который соответствует минимальному SA-WER

    # Инициализация матрицы для подсчета расстояния между словами
    distance_matrix = [[0] * (len(recognized_words) + 1) for _ in range(len(reference_words) + 1)]
    # Наполнение первой строки матрицы
    for i in range(len(reference_words) + 1):
        distance_matrix[i][0] = i

    # Наполнение первого столбца матрицы
    for j in range(len(recognized_words) + 1):
        distance_matrix[0][j] = j

    #print(distance_matrix)
    
    # Заполнение матрицы расстояний методом динамического программирования
    for i in range(1, len(reference_words) + 1):
        for j in range(1, len(recognized_words) + 1):
            if reference_words[i-1] == recognized_words[j-1]:
                cost = 0
            else:
                cost = 1
            distance_matrix[i][j] = min(distance_matrix[i - 1][j] + 1, 
                                        distance_matrix[i][j - 1] + 1, 
                                        distance_matrix[i - 1][j - 1] + cost) 

    wer = distance_matrix[-1][-1]*100/len(reference_words)
    
    correct, deletion, insertion, substitution = 0, 0, 0, 0
    ali = [[],[],[]]
    #print(tabulate(ali))
    #k = fin_length-1
    i = len(reference_words)
    j = len(recognized_words)
    
    while i > 0 or j > 0:
        min_dis = distance_matrix[i][j]
        
        if reference_words[i-1] == recognized_words[j-1] and (i>0 and j>0):
            correct += 1
            ali[2].append('C')
            ali[0].append(reference_words[i-1])
            ali[1].append(recognized_words[j-1])
            i -= 1
            j -= 1
            
        elif (distance_matrix[i - 1][j-1] + 1 == min_dis) and (i>0 and j>0):
            substitution += 1
            ali[2].append('S')
            ali[0].append(reference_words[i-1])
            ali[1].append(recognized_words[j-1])
            i -= 1
            j -= 1
            
        elif (distance_matrix[i - 1][j] + 1 == min_dis) and i>0:
            deletion += 1
            ali[2].append('D')
            ali[0].append(reference_words[i - 1])
            ali[1].append('***')
            i -= 1
        
        else:
            insertion +=1
            ali[2].append('I')
            ali[0].append('***')
            ali[1].append(recognized_words[j-1])
            j -= 1
        #print(i, j)
        #print(ali)

    ali = [list(reversed(i)) for i in ali]

    # коллеги используют https://docs.python.org/3/library/itertools.html#itertools.permutations
    import itertools
    
    ref_speakers_uniq = list(set(reference_speakers)) #уникальные спикеры из рефа
    #print(ref_speakers_uniq)
    rec_speakers_uniq = list(set(recognized_speakers)) #уникальные спикеры из распознавания
    #print(rec_speakers_uniq)
    
    permutations = itertools.permutations(rec_speakers_uniq) #itertools.permutations object all possible full-length permutations of elements in the iterable
    #print(permutations)
    
    SI = len(ref_speakers_uniq)*len(rec_speakers_uniq)*10

    for perm in permutations: # (1, 2) (2, 1)
        errors = 0 # начинаем с 0 ошибок
        #print(perm)
        for i, reference in enumerate(reference_speakers): # i-тое слово принадлежит спикеру reference
            #print(i, reference)#0 A #1 B
            if ref_speakers_uniq.index(reference)+1 < len(perm):
                #print(ref_speakers_uniq.index(reference))
                #print(len(perm))
                if perm[ref_speakers_uniq.index(reference)] != recognized_speakers[i]: 
            # положение reference в ref_speakers_uniq != номеру спикера
                    errors += 1

        if errors < SI:
            SI = errors
    #print(SI)
    #sawer = (insertion + deletion + substitution + SI)/(deletion + substitution + correct + len(reference_speakers))*100
    sawer = wer+SI*100/len(reference_speakers)
    #print(sawer)
    return {"sawer" : sawer, 
            "ali": ali}


In [338]:
calculate_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1])

{'sawer': 50.0, 'ali': [['привет', 'с'], ['привет', '***'], ['C', 'D']]}

In [339]:
def assert_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers, ideal_report):
    report = calculate_sawer(reference_text, reference_speakers, recognized_text, recognized_speakers)
    for k, v in ideal_report.items():
        assert v == report[k]

    
def test_sawer():
    assert_sawer(['привет', 'студент'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 0})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'студент'], ['A', 'A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 0})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет', 'студент'], [1, 2],  {
            "sawer" : 50})
    assert_sawer(['привет', 'с'], ['A', 'B'], ['привет'], [1],  {
            "sawer" : 50})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [1, 0],  {
            "sawer" : 100})
    assert_sawer(['привет'], ['A'], ['привет', 'студент'], [0, 0],  {
            "sawer" : 100})

    print(f"Test 3 passed")
    
test_sawer()

Test 3 passed
